In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

## Import data

In [2]:
df=pd.read_csv('Final_Dataset.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3598607 entries, 0 to 3598606
Data columns (total 13 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Unnamed: 0        int64 
 1   date_             object
 2   user_id_maped     int64 
 3   program_name      object
 4   duration_seconds  int64 
 5   program_class     object
 6   season            int64 
 7   episode           int64 
 8   program_desc      object
 9   program_genre     object
 10  series_title      int64 
 11  hd                int64 
 12  original_name     object
dtypes: int64(7), object(6)
memory usage: 356.9+ MB


In [4]:
# Creating a function which will remove extra leading
# and tailing whitespace from the data.
# pass dataframe as a parameter here
def whitespace_remover(dataframe):
   
    # iterating over the columns
    for i in dataframe.columns:
         
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
             
            # applying strip function on column
            dataframe[i] = dataframe[i].str.strip()
        else:
             
            # if condn. is False then it will do nothing.
            pass

Strip the white space before and after strings

In [5]:
whitespace_remover(df)

Number of unique useres

In [6]:
df.user_id_maped.nunique()

29487

Number of unique MOVIEs & SERIES

In [7]:
df[['program_class','original_name']].groupby(by='program_class').nunique()

,original_name
program_class,
MOVIE,1525
SERIES/EPISODES,276


if series_title = 1 then there is no episode name, the stats for the season itself

In [8]:
df[["year","month","day"]] = df["date_"].str.split("-", expand = True)

In [9]:
df=df.drop(['Unnamed: 0','date_'], axis=1)

In [10]:
df.rename(columns = {'user_id_maped':'user_id','original_name':'movie/series_name',
                     'program_name':'movie/episode_name'}, inplace = True)
df = df[[ 'year','month','day','user_id','movie/series_name','season','episode','movie/episode_name',
         'duration_seconds','program_class','program_genre','series_title','hd','program_desc',]]


In [11]:
df_1=df[df.series_title == 1] # only seasons

In [12]:
df_0=df[df.series_title == 0] # Movies and EPISODES

In [13]:
df_0.duration_seconds.describe()

count    3.555946e+06
mean     1.191115e+03
std      6.266378e+03
min      2.000000e+00
25%      6.000000e+01
50%      1.940000e+02
75%      1.329000e+03
max      2.053603e+06
Name: duration_seconds, dtype: float64

we have outliers in the duration_seconds we have to clean it 

any recored with duration_seconds more than 2.4 hours will be deleted 

In [14]:
df_0=df_0[df_0.duration_seconds<8640]

In [15]:
#sns.histplot(data=df_0, x="duration_seconds",hue='program_class')

In [16]:
df_0.duration_seconds.value_counts()[df_0.duration_seconds.value_counts()>30]

90      47811
6       44522
4       40623
2       40178
5       39780
        ...  
4736       31
5377       31
4128       31
5170       31
5432       31
Name: duration_seconds, Length: 5721, dtype: int64

In [17]:
df_0=df_0[df_0.duration_seconds<4000]

In [18]:
#sns.displot(data=df_0, x="duration_seconds",col='program_class',kind='hist');

movie duration_seconds can not be used to find the run time so I will choose 1 hour and 20 minutes as average runtime for movies

In [19]:
#sns.boxplot(y="duration_seconds",x='program_class',data=df_0)

### Runtime for each Movie/Series

In [20]:
df_0['runtime']=df_0.groupby(by=['movie/series_name','season'])['duration_seconds'].transform(lambda x: x.quantile(.9))

### Numbrer of Views per Movie/Episode

In [21]:
df_0['number_views']=df_0.groupby(['movie/series_name','season','episode'])['duration_seconds'].transform(lambda x: x.count())

In [22]:
df_0.loc[((df_0.program_class == "MOVIE")&(df_0.runtime < 600)|(df_0.number_views < 30)), 'runtime'] = 4800

In [23]:
df_0.shape

(3397189, 16)

### Create Like_it? 

In [24]:
df_0['Like_it?']=df_0.duration_seconds/df_0.runtime

In [25]:
def map_value(x): 
    if (x >= 0.75):
        return 1
    else:
        return 0

df_0['Like_it?'] = df_0['Like_it?'].apply(map_value)

In [26]:
df_0.head()

,year,month,day,user_id,movie/series_name,season,episode,movie/episode_name,duration_seconds,program_class,program_genre,series_title,hd,program_desc,runtime,number_views,Like_it?
0,2017,05,27,26138,100 treets,0,0,100 treets,40,MOVIE,Drama,0,0,Drama Movie100 Streets,4800.0,3756,0
1,2017,05,21,7946,Moana,0,0,Moana,17,MOVIE,Animation,0,1,Animation MovieMoana (HD),4800.0,23903,0
2,2017,08,10,7418,The Mermaid Princess,0,0,The Mermaid Princess,8,MOVIE,Animation,0,1,Animation MovieThe Mermaid Princess (HD),4800.0,21167,0
3,2017,07,26,19307,The Mermaid Princess,0,0,The Mermaid Princess,76,MOVIE,Animation,0,1,Animation MovieThe Mermaid Princess (HD),4800.0,21167,0
4,2017,07,07,15860,Churchill,0,0,Churchill,87,MOVIE,Biography,0,1,Biography MovieChurchill (HD),4800.0,2857,0


In [27]:
#df_0.program_genre.value_counts()

In [28]:
df_0

,year,month,day,user_id,movie/series_name,season,episode,movie/episode_name,duration_seconds,program_class,program_genre,series_title,hd,program_desc,runtime,number_views,Like_it?
0,2017,05,27,26138,100 treets,0,0,100 treets,40,MOVIE,Drama,0,0,Drama Movie100 Streets,4800.0,3756,0
1,2017,05,21,7946,Moana,0,0,Moana,17,MOVIE,Animation,0,1,Animation MovieMoana (HD),4800.0,23903,0
2,2017,08,10,7418,The Mermaid Princess,0,0,The Mermaid Princess,8,MOVIE,Animation,0,1,Animation MovieThe Mermaid Princess (HD),4800.0,21167,0
3,2017,07,26,19307,The Mermaid Princess,0,0,The Mermaid Princess,76,MOVIE,Animation,0,1,Animation MovieThe Mermaid Princess (HD),4800.0,21167,0
4,2017,07,07,15860,Churchill,0,0,Churchill,87,MOVIE,Biography,0,1,Biography MovieChurchill (HD),4800.0,2857,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3598602,2017,08,22,10486,Table,0,0,Table,8,MOVIE,Comedy,0,1,Comedy MovieTable 19 (HD),4800.0,5205,0
3598603,2017,08,22,10486,Newsies: the Broadway Musical,0,0,Newsies: the Broadway Musical,3,MOVIE,Family,0,1,Family MovieNewsies: the Broadway Musical (HD),4800.0,372,0
3598604,2018,01,31,3674,Dragon Ball Z: Resurrection F,0,0,Dragon Ball Z: Resurrection F,7,MOVIE,Animation,0,0,Animation MovieDragon Ball Z: Resurrection F,4800.0,8312,0
3598605,2017,09,19,717,Table,0,0,Table,97,MOVIE,Comedy,0,1,Comedy MovieTable 19 (HD),4800.0,5205,0


## apriori

In [29]:
from apyori import apriori

In [30]:
a=df_0[(df_0['Like_it?']== 1)].groupby(['user_id', 'movie/series_name'])[['Like_it?']].sum().reset_index()

In [31]:
a.head()

,user_id,movie/series_name,Like_it?
0,5,In the Night Garden,6
1,5,Oscar's Oasis,7
2,5,Pingu,13
3,5,Teletubbies,29
4,8,Youm Maloosh Lazma,1


In [32]:
a.shape

(74815, 3)

In [33]:
transactions = []
listt=[]
for i in range(0, a.shape[0]-1):
    listt.append(str(a.values[i, 1]))
    if (a.values[i, 0]) != (a.values[i+1, 0]):
            transactions.append(listt)
            listt=[]

In [37]:
rule = apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3,max_length= 2)

In [38]:
output = list(rule) # returns a non-tabular output
# putting output into a pandas dataframe
def inspect(output):
    lhs         = [tuple(result[2][0][0])[0] for result in output]
    rhs         = [tuple(result[2][0][1])[0] for result in output]
    support    = [result[1] for result in output]
    confidence = [result[2][0][2] for result in output]
    lift       = [result[2][0][3] for result in output]
    return list(zip(lhs, rhs, support, confidence, lift))
output_DataFrame = pd.DataFrame(inspect(output), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

In [62]:
listofmovies=list(output_DataFrame.Left_Hand_Side.unique())
listofmovies.sort()

In [63]:
listofmovies

['12 Monkeys',
 '2 Fil-Isaaf  (T)',
 '24',
 '24: Live Another Day',
 '30 Minutes Or Less',
 '47 Ronin',
 '50 First Dates',
 'A tory and a Proverb',
 'A-Sayeda Al Oula',
 'A-Shaqiqatan',
 'Agent Carter',
 'Agents of .H.I.E.L.D.',
 'Akoon Aw La  (T)',
 'Al Bayt Al Kabeer',
 'Al Hala G',
 'Al Hob Al Halal',
 'Al Kasr Al Kabeer',
 'Al Maaa',
 'Allo Marhaba',
 'Almost Famous',
 'American Horror tory',
 'American Odyssey',
 'Angelo Rules',
 'Anger Management',
 'Animal Mechanicals',
 'Ash vs Evil Dead',
 'Atlanta',
 'Automata',
 'Bad Boys',
 'Bad Teacher',
 'Badges of Fury',
 'Bates Motel',
 'Battle Los Angeles',
 'Battleship',
 'Bayn Al Kanayen  (T)',
 'Bayna Laylatin Wa Dohaha',
 'Bean',
 'Beauty and the Beast',
 'Big Daddy',
 'Billions',
 'Bionicle : The Legend Reborn',
 'Black ails',
 'Black-ish  (T)',
 'Blackish',
 'Blank Check',
 'Blaze And The Monster Machines',
 'Blitz',
 'Bloodline',
 'Bob Al-Banaa',
 'Borj Al Oloum',
 'Bou Droosh',
 'Britannia',
 'Bubble Guppies',
 'Caillou',
 'Car

In [49]:
def movie_recommendation_engine (movie_name):
    rd=output_DataFrame[output_DataFrame.Left_Hand_Side==str(movie_name)].nlargest(n = 10, columns = 'Lift')
    rd.rename(columns = {'Left_Hand_Side':'Your Movie/Series','Right_Hand_Side':'Recommended Movie/Series',
                     'Lift':'Similarity'}, inplace = True)
    return rd[['Your Movie/Series','Recommended Movie/Series','Similarity']].reset_index(drop=True)

In [50]:
movie_recommendation_engine('Dora The Explorer')

,Your Movie/Series,Recommended Movie/Series,Similarity
0,Dora The Explorer,My Little Pony: Friendship Is Magic,10.937319
1,Dora The Explorer,Littlest Pet hop,8.654469
2,Dora The Explorer,Dora the Explorer,7.670854
3,Dora The Explorer,SpongeBob quarePants,7.111184
4,Dora The Explorer,PAW Patrol,7.070125
5,Dora The Explorer,Teenage Mutant Ninja Turtles,6.533154
6,Dora The Explorer,The Thundermans,6.473393
7,Dora The Explorer,Shimmer And hine,6.421112
8,Dora The Explorer,Sonic Underground,6.155230
9,Dora The Explorer,The Penguins of Madagascar,5.852762


In [72]:
movie_recommendation_engine('House')

,Your Movie/Series,Recommended Movie/Series,Similarity
0,House,How I Met Your Mother,8.498701
1,House,Unforgettable,7.225616
2,House,Mr. Robot,6.466403
3,House,Friends,5.068448
4,House,The Flash,4.365626
5,House,Supergirl,4.259843
6,House,Blindspot,3.971694
7,House,The Walking Dead,3.042149


In [70]:
movie_recommendation_engine('Jurassic Park')

,Your Movie/Series,Recommended Movie/Series,Similarity
0,Jurassic Park,Jurassic Park III,12.233056
1,Jurassic Park,ParaNorman,5.111504
2,Jurassic Park,The murfs,4.470271
3,Jurassic Park,Cloudy With a Chance of Meatballs,4.300071
4,Jurassic Park,The Amazing pider-Man,4.008391
5,Jurassic Park,Bean,4.007984
6,Jurassic Park,Teenage Mutant Ninja Turtles,3.636586
7,Jurassic Park,The Penguins of Madagascar,3.430249
8,Jurassic Park,The Thundermans,3.341594
9,Jurassic Park,Blaze And The Monster Machines,3.300043


In [64]:
movie_recommendation_engine('A-Sayeda Al Oula')

,Your Movie/Series,Recommended Movie/Series,Similarity
0,A-Sayeda Al Oula,Wannous,11.346703
1,A-Sayeda Al Oula,Ezai E-Sehha,10.061213
2,A-Sayeda Al Oula,Kalam A-Nas,4.167152


In [65]:
movie_recommendation_engine('Omena Rowehet El Janna')

,Your Movie/Series,Recommended Movie/Series,Similarity
0,Omena Rowehet El Janna,Sadeeqat Al Omor (T),14.455581
1,Omena Rowehet El Janna,Laqet Rohey,11.071509
2,Omena Rowehet El Janna,Bayna Laylatin Wa Dohaha,10.492821
3,Omena Rowehet El Janna,Kalam A-Nas,4.555975
4,Omena Rowehet El Janna,Tom And Jerry: anta's Little Helpers,3.738005
5,Omena Rowehet El Janna,Marcus Level (T),3.656429
6,Omena Rowehet El Janna,Teen Wolf,3.502212
7,Omena Rowehet El Janna,Littlest Pet hop (T),3.453754
8,Omena Rowehet El Janna,My Little Pony: Friendship Is Magic (T),3.420197
9,Omena Rowehet El Janna,Jelly Jamm (T),3.387645


In [67]:
movie_recommendation_engine('The World Wars')

,Your Movie/Series,Recommended Movie/Series,Similarity
0,The World Wars,WW1: The First Modern War,37.785919
1,The World Wars,Vikings,3.550532
